# Decision Trees

Leitura dos datasets tratados

In [1]:
import lightgbm as lgb
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)

## Construção e modelação

In [2]:
label_encoder = preprocessing.LabelEncoder() 

treino['Date']= label_encoder.fit_transform(treino['Date'])
teste['Date']= label_encoder.fit_transform(teste['Date'])

treino['Month']= label_encoder.fit_transform(treino['Month'])
teste['Month']= label_encoder.fit_transform(teste['Month'])

treino['Season']= label_encoder.fit_transform(treino['Season'])
teste['Season']= label_encoder.fit_transform(teste['Season'])

treino['weather_description']= label_encoder.fit_transform(treino['weather_description'])
teste['weather_description']= label_encoder.fit_transform(teste['weather_description'])

In [3]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

In [4]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': list(range(1,20)), 
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.05, 0.1, 0.3]
}

grid = GridSearchCV(lgb.LGBMClassifier(), param_grid, cv=3)
grid.fit(X_treino, y_treino)

grid.best_score_, grid.best_params_

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003997 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2495
[LightGBM] [Info] Number of data points in the train set: 7344, number of used features: 15
[LightGBM] [Info] Start training from score -2.300409
[LightGBM] [Info] Start training from score -3.238678
[LightGBM] [Info] Start training from score -2.305858
[LightGBM] [Info] Start training from score -0.348307
[LightGBM] [Info] Start training from score -2.900224
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000474 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2479
[LightGBM] [Info] Number of data points in the train set: 7344, number of used feature

(0.8388707334785765,
 {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 50})

In [5]:
predictions = grid.predict(X_teste)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/lightBGMPrediction.csv')